Committing to a branch in Git is super straightforward! Here's how it works step-by-step:

---

### ✅ **1. Make sure you're on the right branch**

Check your current branch:
```bash
git branch
```
If you're not on the branch you want to commit to (e.g., `feature/login`), switch to it:

```bash
git checkout feature/login
```

---

### ✅ **2. Make changes to files**

Edit or add files. For example:

```bash
echo "console.log('login page');" > Application/login.js
```

---

### ✅ **3. Stage your changes**

You need to "stage" the changes before committing:

```bash
git add .
```

Or stage a specific file:

```bash
git add Application/login.js
```

---

### ✅ **4. Commit your changes**

Now create a commit with a descriptive message:

```bash
git commit -m "Add login page functionality"
```

This commit is now part of the **current branch only** — it does *not* affect `main` or any other branches.

---

### 🔄 **(Optional) 5. Push to remote branch (e.g., GitHub)**

```bash
git push origin feature/login
```

If it's the first time pushing this branch:

```bash
git push -u origin feature/login
```

---

Let me know if you want to merge this branch later, delete it, or open a pull request!